In [1]:
from __future__ import print_function
import sys,os,datetime
import cv2 as cv
import cv2 as cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import sklearn.metrics as skm
import csv

In [2]:
print('openCV',cv.__version__)
print('numpy',np.__version__)

openCV 3.2.0
numpy 1.13.3


In [19]:
dirTrain1 = '../Data/train/train/level-1'
dirTrain2 = '../Data/train/train/level-2'
dirTrain3 = '../Data/train/train/level-3'
dirTrain  = dirTrain2
dirDev    = '../Data/dev/dev'
dirTest   = '../Data/flat_test/flat_test'
dirForSave = '../Work'

In [4]:
shape2D = (416,416)

In [5]:
def NNTrain(dirTrain, ends='.txt') :
    listTrain= os.listdir(dirTrain); len(listTrain); listTrain[:12]
    listTrainNN = []
    for ff in listTrain :
        if ff.lower().endswith(ends) : listTrainNN.append(ff.split('_')[0])
    return(listTrainNN)
listTrainNN = NNTrain(dirTrain)
len(listTrainNN), listTrainNN[:15]

(1495,
 ['1003',
  '1009',
  '100',
  '1012',
  '1018',
  '1021',
  '984',
  '994',
  '995',
  '997',
  '9',
  '952',
  '955',
  '956',
  '959'])

In [7]:
# Build statistics on masks
img1 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'.tif'),-1)
img2 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'_ref.tif'),-1)
img3 = cv.imread(os.path.join(dirTrain,listTrainNN[0]+'_mask.tif'),-1)
if 0 :
    for dirTrain in (dirTrain1,dirTrain2,dirTrain3) :
        listTrainNN = NNTrain(dirTrain)
        maskSize = []
        for NN in range(len(listTrainNN)) :
            #print(NN)
            img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
            img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)
            img3 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_mask.tif'),-1)
            maskSize.append(img3[img3>0.5].size)
            #img1      = cv.GaussianBlur(img1,(3,3),0)
            #img2      = cv.GaussianBlur(img2,(3,3),0)
        print(img1.shape,img2.shape,img3.shape, dirTrain)

        maskSize = np.array(maskSize)
        print(len(maskSize), len(maskSize[maskSize==0]), maskSize.mean(), np.median(maskSize), maskSize.std(), maskSize.mean()/(390*390)*100.0, float(maskSize.max())/(390*390)*100.0)
        plt.hist(maskSize,bins=150); plt.show()

In [8]:
def diffPicts (img1, img2) :
    diff = []
    for cc in range(img1.shape[2]) :
        ccc   = np.abs(img1[:,:,cc]-img2[:,:,cc])
        cccL2 = np.sqrt(np.square(ccc).mean())
        cccL1 = ccc.max()
        diff.append([cccL1,cccL2,ccc])
    diffL2 = np.zeros(ccc.shape)
    for L1,L2,ccc in diff : diffL2 += np.square(ccc)
    diffL2 = np.sqrt(diffL2)
    return(diff,diffL2)

def minDeltaRC (img1,img2, ranging=range(-5,6), printOK=False) :
    r0,c0 = (img2.shape[0]-img1.shape[0])/2, (img2.shape[1]-img1.shape[1])/2
    minL2 = [390*390*250.0,0,0]
    for dr in ranging :
        for dc in ranging :
            imgX2 = img2[r0+dr:r0+img1.shape[0]+dr,c0+dc:c0+img1.shape[1]+dc,:]
            diff, L2 = diffPicts(img1,imgX2)
            if (minL2[0]>L2.mean()) : minL2 = (L2.mean(),dr,dc,r0+dr,c0+dc,r0,c0)
            if printOK : print (dr,dc,'\t',minL2[0],'\t',L2.max(),L2.mean(),L2.std(),np.median(L2))
    return(minL2)

print('min=',minDeltaRC(img1,img2))

min= (10.974686533082986, 0, 0, 55, 55, 55, 55)


In [9]:
def pictNorm (img255, std=False) :
    imgF32 = np.float32(img255) 
    for cc in range(img255.shape[2]) :
        imgF32[:,:,cc] -= imgF32[:,:,cc].mean()
        if std : imgF32[:,:,cc] /= imgF32[:,:,cc].std()**2
        imgF32[:,:,cc] /= 127.0
    return(imgF32)

In [10]:
def computeORB (img, debug=False) :
    orb    = cv2.ORB_create();
    kp     = orb.detect(img,None)
    kp,des = orb.compute(img,kp)
    if debug :
        print (len(kp))
        print (len(des),len(des[0]),"\n",des[0],"\n",des[1])
    return kp,des

def computeMatches (des1,des2) :
    if True :
        bf      = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
        matches = bf.match(des1,des2);
        #????matches = bf.knnMatch(des1,des2,k=1);
    if False :
        index_params = dict(algorithm = 6, #cv2.FLANN_INDEX_LSH,
                            table_number = 6, 
                            key_size = 12,    
                            multi_probe_level = 1)
        search_params = dict(checks=100)
        flann   = cv2.FlannBasedMatcher(index_params,search_params)
        matchesI = flann.knnMatch(des1,des2,k=3);
        matches  = [];
        for mI in matchesI :
            for m in mI : matches.append(m);
    return matches    

def drawMatches(img1,kp1,des1,img2,kp2,des2, show=False, showNN=20):
    matches = computeMatches(des1,des2);
    matches = sorted(matches,key=lambda x:x.distance)
    if  show :
        imgM    = img1.copy()
        #imgM    = cv2.drawMatches(img1,kp1,img2,kp2,matches[:20],imgM,flags=4)
        imgM    = cv2.drawMatches(img1,kp1,img2,kp2,matches[:showNN],imgM,flags=2) #4)
        #imgM    = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,imgM,flags=2) #4)
        
        plt.figure(figsize=(15,15)); plt.imshow(imgM), plt.show()
    return(matches)


In [11]:
len(listTrainNN)

1495

In [ ]:
print(datetime.datetime.now())

for ii,dirTrain in enumerate((dirTrain1,dirTrain2,dirTrain3)) :
    
    #if ii==1 : break
    
    listTrainNN = NNTrain(dirTrain)

    X1 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    YY = np.zeros((len(listTrainNN),)+shape2D+(1,),dtype=np.float32)
    print(datetime.datetime.now(),X1.shape, X2.shape,YY.shape,dirTrain)
    
    for NN in range(len(listTrainNN)) :
        img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
        img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)
        img3 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_mask.tif'),-1); img3 = img3.reshape(img3.shape+(1,))



        if 1 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2 = cv2.GaussianBlur(img2,(3,3),0)

        if 0 :
            img1 = cv2.boxFilter(img1,-1,(5,5))
            img2 = cv2.boxFilter(img2,-1,(5,5))


        if 1 :
            img1  = pictNorm(img1)
            img2  = pictNorm(img2)

        rcL2,_,_,r0,c0,ro,co  = minDeltaRC(img1,img2,ranging=range(-10,10))
        img2X = img2[r0:r0+img1.shape[0],c0:c0+img1.shape[1]]
        ##########img2X = np.array(img2[ro:(ro+img1.shape[0]),co:co+img1.shape[1]])
        
        #print(img2[ro:ro+img1.shape[0],co:co+img1.shape[1]].shape)
        #print(img2.shape,img2X.shape,ro,ro+img1.shape[0],co,co+img1.shape[1])
        
        if NN%100==0 : print(datetime.datetime.now(),NN,rcL2,ro,co,r0,c0)

        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2X = cv2.GaussianBlur(img2X,(3,3),0)

        if 1 :
            img1  = cv2.boxFilter(img1, -1,(5,5))
            img2X = cv2.boxFilter(img2X,-1,(5,5))

        X1[NN,13:13+390,13:13+390,:], X2[NN,13:13+390,13:13+390,:], YY[NN,13:13+390,13:13+390,:] = img1, img2X, img3


    if 1 :
        np.save(os.path.join(dirForSave,'train-'+str(ii)+'-NN-101D01.npy'),np.array(listTrainNN))
        np.save(os.path.join(dirForSave,'train-'+str(ii)+'-X1-101D01.npy'),X1)
        np.save(os.path.join(dirForSave,'train-'+str(ii)+'-X2-101D01.npy'),X2)
        np.save(os.path.join(dirForSave,'train-'+str(ii)+'-YY-101D01.npy'),YY)
        print(datetime.datetime.now(),'Save:',X1.shape, X2.shape,YY.shape,dirTrain)
        
    del X1,X2,YY
        
print(datetime.datetime.now())

2018-02-15 18:19:39.534435
2018-02-15 18:19:39.549933 (751, 416, 416, 3) (751, 416, 416, 3) (751, 416, 416, 1) ../Data/train/train/level-1
2018-02-15 18:19:40.504342 0 0.0658074848063 55 55 55 56
2018-02-15 18:21:10.458308 100 0.0567938326306 55 55 56 56
2018-02-15 18:22:43.908146 200 0.0676420703097 55 55 56 53
2018-02-15 18:24:16.749213 300 0.0566413581779 55 55 54 50
2018-02-15 18:25:46.733279 400 0.0549176484386 55 55 46 64
2018-02-15 18:27:18.568629 500 0.0510736484509 55 55 54 56
2018-02-15 18:28:56.069417 600 0.0628774579128 55 55 55 55
2018-02-15 18:30:23.589734 700 0.205298348405 55 55 55 52
2018-02-15 18:31:51.924044 Save: (751, 416, 416, 3) (751, 416, 416, 3) (751, 416, 416, 1) ../Data/train/train/level-1
2018-02-15 18:31:52.061104 (1495, 416, 416, 3) (1495, 416, 416, 3) (1495, 416, 416, 1) ../Data/train/train/level-2
2018-02-15 18:31:53.016554 0 0.135193295812 55 55 55 55
2018-02-15 18:33:37.962201 100 0.110332033177 55 55 55 55
2018-02-15 18:35:25.166285 200 0.044688557428

In [26]:
if 0 :
    np.save(os.path.join(dirForSave,'X1.npy'),X1)
    np.save(os.path.join(dirForSave,'X2.npy'),X2)
    np.save(os.path.join(dirForSave,'YY.npy'),YY)
    
if 0 :
    ii = 0
    X1 = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'.npy'))
    X2 = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'.npy'))
    YY = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
    print(datetime.datetime.now(),'Load: ',ii,X1.shape, X2.shape,YY.shape)
    
if 0 :
    iil = []
    for ii in (0,1,2) :
        YY = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        iil.append(len(YY))
    iis = int(np.array(iil).sum())
    
    X1 = np.zeros((iis,)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((iis,)+shape2D+(3,),dtype=np.float32)
    YY = np.zeros((iis,)+shape2D+(1,),dtype=np.float32)
    
    iis = 0
    
    for ii in (0,1,2) :
        
        T1  = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'.npy'))
        T2  = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'.npy'))
        TT  = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        
        X1[iis:iis+len(TT)], X2[iis:iis+len(TT)], YY[iis:iis+len(TT)] = T1, T2, TT
        
        iis+= len(TT)
        
        del TT,T1, T2
        
    print(datetime.datetime.now(),'Load: ', X1.shape, X2.shape, YY.shape)
    

In [ ]:
# Build Devs arrays
print(datetime.datetime.now())

for ii,dirTrain in enumerate((os.path.join(dirDev,'level-1'),
                              os.path.join(dirDev,'level-2'),
                              os.path.join(dirDev,'level-3'))) :
    
    #if ii==1 : break
    
    listTrainNN = NNTrain(dirTrain,ends='_ref.tif')

    X1 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    X2 = np.zeros((len(listTrainNN),)+shape2D+(3,),dtype=np.float32)
    ##YY = np.zeros((len(listTrainNN),)+shape2D+(1,),dtype=np.float32)
    print(datetime.datetime.now(),X1.shape, X2.shape,dirTrain)
    
    for NN in range(len(listTrainNN)) :
        img1 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'.tif'),-1)
        img2 = cv.imread(os.path.join(dirTrain,listTrainNN[NN]+'_ref.tif'),-1)



        if 1 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2 = cv2.GaussianBlur(img2,(3,3),0)

        if 0 :
            img1 = cv2.boxFilter(img1,-1,(5,5))
            img2 = cv2.boxFilter(img2,-1,(5,5))


        if 1 :
            img1  = pictNorm(img1)
            img2  = pictNorm(img2)

        rcL2,_,_,r0,c0,ro,co  = minDeltaRC(img1,img2,ranging=range(-10,10))
        img2X = img2[r0:r0+img1.shape[0],c0:c0+img1.shape[1]]
        #####################img2X = np.array(img2[ro:(ro+img1.shape[0]),co:co+img1.shape[1]])
        
        #print(img2[ro:ro+img1.shape[0],co:co+img1.shape[1]].shape)
        #print(img2.shape,img2X.shape,ro,ro+img1.shape[0],co,co+img1.shape[1])
        
        if NN%100==0 : print(datetime.datetime.now(),NN,rcL2,ro,co,r0,c0)

        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2X = cv2.GaussianBlur(img2X,(3,3),0)

        if 1 :
            img1  = cv2.boxFilter(img1, -1,(5,5))
            img2X = cv2.boxFilter(img2X,-1,(5,5))

        X1[NN,13:13+390,13:13+390,:], X2[NN,13:13+390,13:13+390,:] = img1, img2X


    if 1 :
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-NN-101D01.npy'),np.array(listTrainNN))
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-X1-101D01.npy'),X1)
        np.save(os.path.join(dirForSave,'dev-'+str(ii)+'-X2-101D01.npy'),X2)
        print(datetime.datetime.now(),'Save:',X1.shape, X2.shape,dirTrain)
        
    del X1,X2
        
print(datetime.datetime.now())

2018-02-15 19:53:26.709370
2018-02-15 19:53:30.418989 (245, 416, 416, 3) (245, 416, 416, 3) ../Data/dev/dev/level-1
2018-02-15 19:53:36.272750 0 0.0960723423375 55 55 54 55
2018-02-15 19:55:29.668277 100 0.0592844895718 55 55 56 55
2018-02-15 19:57:18.553652 200 0.0870139376873 55 55 63 51
2018-02-15 19:58:12.015671 Save: (245, 416, 416, 3) (245, 416, 416, 3) ../Data/dev/dev/level-1
2018-02-15 19:58:12.929642 (510, 416, 416, 3) (510, 416, 416, 3) ../Data/dev/dev/level-2
2018-02-15 19:58:14.276635 0 0.0694949326184 55 55 54 55
2018-02-15 20:00:06.041294 100 0.105673297447 55 55 54 55
2018-02-15 20:02:02.677059 200 0.129993490945 55 55 54 55
2018-02-15 20:09:15.298235 300 0.118333986122 55 55 56 56
2018-02-15 20:13:46.070069 400 0.060067258166 55 55 54 55
2018-02-15 20:18:40.976057 500 0.0589160094272 55 55 55 55
2018-02-15 20:19:39.830261 Save: (510, 416, 416, 3) (510, 416, 416, 3) ../Data/dev/dev/level-2
2018-02-15 20:19:42.842772 (650, 416, 416, 3) (650, 416, 416, 3) ../Data/dev/dev/l

In [ ]:
# Train build 3+3=6

In [20]:
 if 1 :
    print(datetime.datetime.now(),'For:  ')
    iil = []
    for ii in (0,1,2) :
        YY = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        iil.append(len(YY))
    iis = int(np.array(iil).sum())
    
    NN = np.array(['1234']*iis)
    XX = np.zeros((iis,)+shape2D+(6,),dtype=np.float32)
    YY = np.zeros((iis,)+shape2D+(1,),dtype=np.float32)
    
    iis = 0
    
    for ii in (0,1,2) :

        if 0 :
            TN  = np.load(os.path.join(dirForSave,'NN-'+str(ii)+'S.npy'))
            T1  = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'.npy'))
            T2  = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'.npy'))
            TT  = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'.npy'))
        
        if 0 : # 2018-02-15
            TN  = np.load(os.path.join(dirForSave,'NN-'+str(ii)+'S.npy'))
            T1  = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'S.npy'))
            T2  = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'S.npy'))
            TT  = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'S.npy'))
        
        if 1 : # 2018-02-21
            TN  = np.load(os.path.join(dirForSave,'NN-'+str(ii)+'S.npy'))
            T1  = np.load(os.path.join(dirForSave,'X1-'+str(ii)+'S.npy'))
            T2  = np.load(os.path.join(dirForSave,'X2-'+str(ii)+'S.npy'))
            TT  = np.load(os.path.join(dirForSave,'YY-'+str(ii)+'S.npy'))
        
        YY[iis:iis+len(TT)], NN[iis:iis+len(TT)] = TT, TN
        XX[iis:iis+len(TT),:,:,0:3]              = T1
        XX[iis:iis+len(TT),:,:,3:6]              = T2
        
        
        iis+= len(TT)
        
        del TT, T1, T2, TN
        
    

    np.save(os.path.join(dirForSave,'train-NN-S6.npy'),NN)
    np.save(os.path.join(dirForSave,'train-YY-S6.npy'),YY)
    np.save(os.path.join(dirForSave,'train-XX-S6.npy'),XX)
    

    print(datetime.datetime.now(),'Save: ', XX.shape, YY.shape, NN.shape)
    
    
    YY = YY/YY.max()
    XX.shape,YY.shape, YY.max()

2018-02-21 02:40:28.706383 For:  
2018-02-21 02:45:58.944787 Save:  (4119, 416, 416, 6) (4119, 416, 416, 1) (4119,)


In [ ]:
# Dev build 3+3=6

In [24]:
 if 1 :
    print(datetime.datetime.now(),'For:  ')
    iil = []
    for ii in (0,1,2) :
        YY = np.load(os.path.join(dirForSave,'dev-'+str(ii)+'-NN-S.npy'))
        iil.append(len(YY))
    iis = int(np.array(iil).sum())
    
    NN = np.array(['1234']*iis)
    XX = np.zeros((iis,)+shape2D+(6,),dtype=np.float32)
    
    iis = 0
    
    for ii in (0,1,2) :

        if 1 : # 2018-02-21
            TN  = np.load(os.path.join(dirForSave,'dev-'+str(ii)+'-NN-S.npy'))
            T1  = np.load(os.path.join(dirForSave,'dev-'+str(ii)+'-X1-S.npy'))
            T2  = np.load(os.path.join(dirForSave,'dev-'+str(ii)+'-X2-S.npy'))
        
        NN[iis:iis+len(TN)]                      = TN
        XX[iis:iis+len(TN),:,:,0:3]              = T1
        XX[iis:iis+len(TN),:,:,3:6]              = T2
        
        
        iis+= len(TN)
        
        del T1, T2, TN
        
    

    np.save(os.path.join(dirForSave,'dev-NN-S6.npy'),NN)
    np.save(os.path.join(dirForSave,'dev-XX-S6.npy'),XX)
    

    print(datetime.datetime.now(),'Save: ', XX.shape, NN.shape)

2018-02-21 02:55:46.457958 For:  
2018-02-21 02:57:00.078377 Save:  (1405, 416, 416, 6) (1405,)


In [ ]:
# Build Test arrays 3+3=6

In [26]:
print(datetime.datetime.now())

for ii,dirs in enumerate((dirTest,)) :
    
    #if ii==1 : break
    
    listTrainNN = NNTrain(dirs,ends='_ref.tif')

    XX = np.zeros((len(listTrainNN),)+shape2D+(6,),dtype=np.float32)
    
    print(datetime.datetime.now(),XX.shape,dirTrain)
    
    for NN in range(len(listTrainNN)) :
        img1 = cv.imread(os.path.join(dirs,listTrainNN[NN]+'.tif'),-1)
        img2 = cv.imread(os.path.join(dirs,listTrainNN[NN]+'_ref.tif'),-1)



        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2 = cv2.GaussianBlur(img2,(3,3),0)

        if 0 :
            img1 = cv2.boxFilter(img1,-1,(5,5))
            img2 = cv2.boxFilter(img2,-1,(5,5))


        if 1 :
            img1  = pictNorm(img1)
            img2  = pictNorm(img2)

        rcL2,_,_,r0,c0,ro,co  = minDeltaRC(img1,img2,ranging=range(-10,10))
        img2X = img2[r0:r0+img1.shape[0],c0:c0+img1.shape[1]]
        #####################img2X = np.array(img2[ro:(ro+img1.shape[0]),co:co+img1.shape[1]])
        
        #print(img2[ro:ro+img1.shape[0],co:co+img1.shape[1]].shape)
        #print(img2.shape,img2X.shape,ro,ro+img1.shape[0],co,co+img1.shape[1])
        
        if NN%100==0 : print(datetime.datetime.now(),NN,rcL2,ro,co,r0,c0)

        if 0 :
            img1 = cv2.GaussianBlur(img1,(3,3),0)
            img2X = cv2.GaussianBlur(img2X,(3,3),0)

        if 0 :
            img1  = cv2.boxFilter(img1, -1,(5,5))
            img2X = cv2.boxFilter(img2X,-1,(5,5))

        XX[NN,13:13+390,13:13+390,0:3], XX[NN,13:13+390,13:13+390,3:6] = img1, img2X


    if 1 :
        np.save(os.path.join(dirForSave,'test-NN-S6.npy'),np.array(listTrainNN))
        np.save(os.path.join(dirForSave,'test-XX-S6.npy'),XX)
        print(datetime.datetime.now(),'Save:',XX.shape,dirTrain)
        
    del XX
        
print(datetime.datetime.now())

2018-02-23 05:27:04.856980
2018-02-23 05:27:04.861051 (1358, 416, 416, 6) D
2018-02-23 05:27:06.396171 0 0.116556978444 55 55 54 56
2018-02-23 05:28:36.279520 100 0.118557774276 55 55 54 54
2018-02-23 05:30:04.127896 200 0.146338524043 55 55 55 56
2018-02-23 05:31:31.714768 300 0.120774868864 55 55 54 59
2018-02-23 05:32:59.223947 400 0.107190340127 55 55 55 55
2018-02-23 05:34:28.406313 500 0.0788800358778 55 55 55 55
2018-02-23 05:35:56.601098 600 0.138193844243 55 55 53 54
2018-02-23 05:37:24.718585 700 0.0771055474025 55 55 55 55
2018-02-23 05:38:52.247799 800 0.239483193502 55 55 53 54
2018-02-23 05:40:19.310520 900 0.125337400771 55 55 55 54
2018-02-23 05:41:46.516347 1000 0.126004212684 55 55 55 53
2018-02-23 05:43:14.250385 1100 0.143666511371 55 55 56 55
2018-02-23 05:44:41.803859 1200 0.123388952189 55 55 55 57
2018-02-23 05:46:09.345255 1300 0.122482899034 55 55 54 54
2018-02-23 05:47:49.542089 Save: (1358, 416, 416, 6) D
2018-02-23 05:47:49.762690
